In [1]:
from fasthtml.common import *

In [2]:
db = database('uplayers.db')
players = db.t.players
if players not in db.t:
    players.create(id=int, name=str, points=int, pk='id')

In [3]:
players

<Table players (id, name, points)>

In [7]:
players.insert({'name': "dsadsa", 'points': 20})

{'id': 2, 'name': 'dsadsa', 'points': 20}

In [17]:
db.q(f"select * from {players} where {players.c.name} like 'dsadsa' limit 1")

[{'id': 1, 'name': 'dsadsa', 'points': 20}]

In [30]:
db.q(f"select * from {players} where {players.c.name} like 'dsadsa' limit 1")[0]['points']

20

In [4]:
players.drop()

In [10]:
list(players.rows)

[{'id': 1, 'name': 'Mihaiii', 'points': 194}]

In [9]:
players

<Table players (id, name, points)>

In [15]:
ids = ", ".join([str(x) for x in list_of_ints])
db.q(f"select * from {players} where {players.c.id} in ({ids})")

[{'id': 1, 'name': 'Mihaiii', 'points': 194}]